This is a random collection of pandas tips and tricks.

### Sections

* [Loading Data](###Loading-Data)
* [Renaming Columns](###Renaming-Columns)
* [Renaming Column Names to Lowercase](####Renaming-Column-Names-to-Lowercase)
* [Renaming Specific Columns](####Renaming-Specific-Columns)
* [Row-wise computations](###Row-wise-computations)
* [Changing Values in a Column](####Changing-Values-in-a-Column)
* [Adding a New Column](####Add-a-New-Column)
* [Applying Functions to Multiple Columns](####Applying-Functions-to-Multiple Columns)
* [Missing Values](###Missing-Values)
* [Counting Rows with NaNs](####Counting-Rows-with-NaNs)
* [Selecting NaN Rows](####Selecting-NaN-Rows)
* [Filling NaN Rows](####Filling-NaN-Rows)
* [Appending Rows to a DataFrame](###Appending-Rows-to-a-DataFrame)
* [Sorting and Reindexing DataFrames](###Sorting-and-Reindexing-DataFrames)
* [Chaining Conditions](###Chaining-Conditions)
* [Column Types](###Column-Types)
* [Printing Column Types](####Printing-Column-Types)
* [Selecting by Column Type](####Selecting-by-Column-Type)
* [Converting Column Types](####Converting-Column-Types)
* [If Tests](###If-Tests)

### Loading Data

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python_reference/master/Data/some_soccer_data.csv')
df

PLAYER  SALARY    GP   G     A  SOT  \
0      Sergio Agüero\n Forward — Manchester City  $19.2m  16.0  14   3.0   34   
1               Eden Hazard\n Midfield — Chelsea  $18.9m  21.0   8   4.0   17   
2             Alexis Sánchez\n Forward — Arsenal  $17.6m   NaN  12   7.0   29   
3        Yaya Touré\n Midfield — Manchester City  $16.6m  18.0   7   1.0   19   
4  Ángel Di María\n Midfield — Manchester United  $15.0m  13.0   3   NaN   13   
5          Santiago Cazorla\n Midfield — Arsenal  $14.8m  20.0   4   NaN   20   
6       David Silva\n Midfield — Manchester City  $14.3m  15.0   6   2.0   11   
7             Cesc Fàbregas\n Midfield — Chelsea  $14.0m  20.0   2  14.0   10   
8           Saido Berahino\n Forward — West Brom  $13.8m  21.0   9   0.0   20   
9          Steven Gerrard\n Midfield — Liverpool  $13.8m  20.0   5   1.0   11   

     PPG       P  
0  13.12  209.98  
1  13.05  274.04  
2  11.19  223.86  
3  10.99  197.91  
4  10.17  132.23  
5   9.97     NaN  
6  10.35  155.26  
7  10.47  209.49  
8   7.02  147.43  
9   7.50  150.01

### Renaming Columns

#### Renaming Column Names to Lowercase

In [2]:
df.columns = [c.lower() for c in df.columns]

df.tail(3)

player  salary    gp  g     a  sot    ppg  \
7     Cesc Fàbregas\n Midfield — Chelsea  $14.0m  20.0  2  14.0   10  10.47   
8   Saido Berahino\n Forward — West Brom  $13.8m  21.0  9   0.0   20   7.02   
9  Steven Gerrard\n Midfield — Liverpool  $13.8m  20.0  5   1.0   11   7.50   

        p  
7  209.49  
8  147.43  
9  150.01

#### Renaming Specific Columns

In [3]:
df = df.rename(columns = {
    'p': 'points',
    'gp': 'games',
    'sot': 'shots_on_target',
    'g': 'goals',
    'ppg': 'points_per_game',
    'a': 'assists'
})

df.tail(3)

player  salary  games  goals  assists  \
7     Cesc Fàbregas\n Midfield — Chelsea  $14.0m   20.0      2     14.0   
8   Saido Berahino\n Forward — West Brom  $13.8m   21.0      9      0.0   
9  Steven Gerrard\n Midfield — Liverpool  $13.8m   20.0      5      1.0   

   shots_on_target  points_per_game  points  
7               10            10.47  209.49  
8               20             7.02  147.43  
9               11             7.50  150.01

### Row-wise computations

#### Changing Values in a Column

In [4]:
# process the `salary` column

df['salary'] = df['salary'].apply(lambda x: x.strip('$m'))

df.tail()

player salary  games  goals  assists  \
5     Santiago Cazorla\n Midfield — Arsenal   14.8   20.0      4      NaN   
6  David Silva\n Midfield — Manchester City   14.3   15.0      6      2.0   
7        Cesc Fàbregas\n Midfield — Chelsea   14.0   20.0      2     14.0   
8      Saido Berahino\n Forward — West Brom   13.8   21.0      9      0.0   
9     Steven Gerrard\n Midfield — Liverpool   13.8   20.0      5      1.0   

   shots_on_target  points_per_game  points  
5               20             9.97     NaN  
6               11            10.35  155.26  
7               10            10.47  209.49  
8               20             7.02  147.43  
9               11             7.50  150.01

#### Add a New Column

In [6]:
df['team'] = pd.Series('', index=df.index)

# OR

df.insert(loc=8, column='position', value='')

df.tail(3)

player salary  games  goals  assists  \
7     Cesc Fàbregas\n Midfield — Chelsea   14.0   20.0      2     14.0   
8   Saido Berahino\n Forward — West Brom   13.8   21.0      9      0.0   
9  Steven Gerrard\n Midfield — Liverpool   13.8   20.0      5      1.0   

   shots_on_target  points_per_game  points position team  
7               10            10.47  209.49                
8               20             7.02  147.43                
9               11             7.50  150.01

In [7]:
# process the `player` column

def process_play_col(text):
    name, rest = text.split('\n')
    position, team = [x.strip() for x in rest.split(' — ')]
    return pd.Series([name, team, position])

df[['player', 'team', 'position']] = df.player.apply(process_play_col)

In [8]:
df.tail()

player salary  games  goals  assists  shots_on_target  \
5  Santiago Cazorla   14.8   20.0      4      NaN               20   
6       David Silva   14.3   15.0      6      2.0               11   
7     Cesc Fàbregas   14.0   20.0      2     14.0               10   
8    Saido Berahino   13.8   21.0      9      0.0               20   
9    Steven Gerrard   13.8   20.0      5      1.0               11   

   points_per_game  points  position             team  
5             9.97     NaN  Midfield          Arsenal  
6            10.35  155.26  Midfield  Manchester City  
7            10.47  209.49  Midfield          Chelsea  
8             7.02  147.43   Forward        West Brom  
9             7.50  150.01  Midfield        Liverpool

#### Applying Functions to Multiple Columns

In [9]:
cols = ['player', 'position', 'team']

df[cols] = df[cols].applymap(lambda x: x.lower())

df.head()

player salary  games  goals  assists  shots_on_target  \
0   sergio agüero   19.2   16.0     14      3.0               34   
1     eden hazard   18.9   21.0      8      4.0               17   
2  alexis sánchez   17.6    NaN     12      7.0               29   
3      yaya touré   16.6   18.0      7      1.0               19   
4  ángel di maría   15.0   13.0      3      NaN               13   

   points_per_game  points  position               team  
0            13.12  209.98   forward    manchester city  
1            13.05  274.04  midfield            chelsea  
2            11.19  223.86   forward            arsenal  
3            10.99  197.91  midfield    manchester city  
4            10.17  132.23  midfield  manchester united

### Missing Values

#### Counting Rows with NaNs

In [10]:
nans = df.shape[0] - df.dropna().shape[0]

print(f'{nans} rows have missing values')

3 rows have missing values


#### Selecting NaN Rows

In [11]:
# Select all rows that have NaNs in the `assists` column

df[df['assists'].isnull()]

player salary  games  goals  assists  shots_on_target  \
4    ángel di maría   15.0   13.0      3      NaN               13   
5  santiago cazorla   14.8   20.0      4      NaN               20   

   points_per_game  points  position               team  
4            10.17  132.23  midfield  manchester united  
5             9.97     NaN  midfield            arsenal

#### Filling NaN Rows

In [12]:
# Fill NaN cells with default value 0

df.fillna(value=0, inplace=True)

df

player salary  games  goals  assists  shots_on_target  \
0     sergio agüero   19.2   16.0     14      3.0               34   
1       eden hazard   18.9   21.0      8      4.0               17   
2    alexis sánchez   17.6    0.0     12      7.0               29   
3        yaya touré   16.6   18.0      7      1.0               19   
4    ángel di maría   15.0   13.0      3      0.0               13   
5  santiago cazorla   14.8   20.0      4      0.0               20   
6       david silva   14.3   15.0      6      2.0               11   
7     cesc fàbregas   14.0   20.0      2     14.0               10   
8    saido berahino   13.8   21.0      9      0.0               20   
9    steven gerrard   13.8   20.0      5      1.0               11   

   points_per_game  points  position               team  
0            13.12  209.98   forward    manchester city  
1            13.05  274.04  midfield            chelsea  
2            11.19  223.86   forward            arsenal  
3            10.99  197.91  midfield    manchester city  
4            10.17  132.23  midfield  manchester united  
5             9.97    0.00  midfield            arsenal  
6            10.35  155.26  midfield    manchester city  
7            10.47  209.49  midfield            chelsea  
8             7.02  147.43   forward          west brom  
9             7.50  150.01  midfield          liverpool

### Appending Rows to a DataFrame

In [14]:
# Add an empty row

import numpy as np

df = df.append(pd.Series(
                [np.nan] * len(df.columns), # fill cells with NaNs
                index = df.columns),
                ignore_index = True)

df.tail(3)

player salary  games  goals  assists  shots_on_target  \
8   saido berahino   13.8   21.0    9.0      0.0             20.0   
9   steven gerrard   13.8   20.0    5.0      1.0             11.0   
10             NaN    NaN    NaN    NaN      NaN              NaN   

    points_per_game  points  position       team  
8              7.02  147.43   forward  west brom  
9              7.50  150.01  midfield  liverpool  
10              NaN     NaN       NaN        NaN

In [15]:
# Filling cells with data

df.loc[df.index[-1], 'player'] = 'new player'
df.loc[df.index[-1], 'salary'] = 12.3

df.tail()

player salary  games  goals  assists  shots_on_target  \
6      david silva   14.3   15.0    6.0      2.0             11.0   
7    cesc fàbregas   14.0   20.0    2.0     14.0             10.0   
8   saido berahino   13.8   21.0    9.0      0.0             20.0   
9   steven gerrard   13.8   20.0    5.0      1.0             11.0   
10      new player   12.3    NaN    NaN      NaN              NaN   

    points_per_game  points  position             team  
6             10.35  155.26  midfield  manchester city  
7             10.47  209.49  midfield          chelsea  
8              7.02  147.43   forward        west brom  
9              7.50  150.01  midfield        liverpool  
10              NaN     NaN       NaN              NaN

### Sorting and Reindexing DataFrames

In [17]:
# Sorting the DataFrame by a certain column (from highest to lowest)

df.sort_values('goals', ascending=False, inplace=True)

df.head()

player salary  games  goals  assists  shots_on_target  \
0   sergio agüero   19.2   16.0   14.0      3.0             34.0   
2  alexis sánchez   17.6    0.0   12.0      7.0             29.0   
8  saido berahino   13.8   21.0    9.0      0.0             20.0   
1     eden hazard   18.9   21.0    8.0      4.0             17.0   
3      yaya touré   16.6   18.0    7.0      1.0             19.0   

   points_per_game  points  position             team  
0            13.12  209.98   forward  manchester city  
2            11.19  223.86   forward          arsenal  
8             7.02  147.43   forward        west brom  
1            13.05  274.04  midfield          chelsea  
3            10.99  197.91  midfield  manchester city

In [18]:
# Reindex a DataFrame after sorting

df.index = range(1, len(df.index)+1)
df.head()

player salary  games  goals  assists  shots_on_target  \
1   sergio agüero   19.2   16.0   14.0      3.0             34.0   
2  alexis sánchez   17.6    0.0   12.0      7.0             29.0   
3  saido berahino   13.8   21.0    9.0      0.0             20.0   
4     eden hazard   18.9   21.0    8.0      4.0             17.0   
5      yaya touré   16.6   18.0    7.0      1.0             19.0   

   points_per_game  points  position             team  
1            13.12  209.98   forward  manchester city  
2            11.19  223.86   forward          arsenal  
3             7.02  147.43   forward        west brom  
4            13.05  274.04  midfield          chelsea  
5            10.99  197.91  midfield  manchester city

### Chaining Conditions

In [21]:
# Select players that either play for Arsenal or Chelsea

df[ (df['team'] == 'arsenal') | (df['team'] == 'chelsea') ]

player salary  games  goals  assists  shots_on_target  \
2     alexis sánchez   17.6    0.0   12.0      7.0             29.0   
4        eden hazard   18.9   21.0    8.0      4.0             17.0   
8   santiago cazorla   14.8   20.0    4.0      0.0             20.0   
10     cesc fàbregas   14.0   20.0    2.0     14.0             10.0   

    points_per_game  points  position     team  
2             11.19  223.86   forward  arsenal  
4             13.05  274.04  midfield  chelsea  
8              9.97    0.00  midfield  arsenal  
10            10.47  209.49  midfield  chelsea

In [22]:
# Select forwards from Arsenal only

df[ (df['team'] == 'arsenal') & (df['position'] == 'forward') ]

player salary  games  goals  assists  shots_on_target  \
2  alexis sánchez   17.6    0.0   12.0      7.0             29.0   

   points_per_game  points position     team  
2            11.19  223.86  forward  arsenal

### Column Types

#### Printing Column Types

In [23]:
types = df.columns.to_series().groupby(df.dtypes).groups

types

{dtype('float64'): Index(['games', 'goals', 'assists', 'shots_on_target', 'points_per_game',
        'points'],
       dtype='object'),
 dtype('O'): Index(['player', 'salary', 'position', 'team'], dtype='object')}

#### Selecting by Column Type

In [25]:
# select string columns

df.loc[:, (df.dtypes == np.dtype('O')).values].head()

player salary  position             team
1   sergio agüero   19.2   forward  manchester city
2  alexis sánchez   17.6   forward          arsenal
3  saido berahino   13.8   forward        west brom
4     eden hazard   18.9  midfield          chelsea
5      yaya touré   16.6  midfield  manchester city

#### Converting Column Types

In [26]:
df['salary'] = df['salary'].astype(float)

In [27]:
types = df.columns.to_series().groupby(df.dtypes).groups

types

{dtype('float64'): Index(['salary', 'games', 'goals', 'assists', 'shots_on_target',
        'points_per_game', 'points'],
       dtype='object'),
 dtype('O'): Index(['player', 'position', 'team'], dtype='object')}

### If Tests

This process creates an array of 1s and 0s depending on a condition. Ex, if `val` is less than 0.5 then 0, else 1.

We can do this using a boolean mask, since `True` and `False` are integers.

In [28]:
int(True)

1

In [29]:
a = [[2., .3, 4., 5.], [.8, .03, 0.02, 5.]]
df = pd.DataFrame(a)

df

0     1     2    3
0  2.0  0.30  4.00  5.0
1  0.8  0.03  0.02  5.0

In [30]:
df = df <= 0.5
df

0     1      2      3
0  False  True  False  False
1  False  True   True  False

In [31]:
df.astype(int)

0  1  2  3
0  0  1  0  0
1  0  1  1  0